Extracting the final parameter set

In [1]:
import os
import numpy as np

In [7]:
# Extract parameter values and losses into np.arrays
def create_list(file_name, log_path):
    if log_path.endswith('auto_tune_iconaml'):
        no_params = 24
    elif log_path.endswith('auto_tune_icona_baseline') or log_path.endswith('icon-a-ml/run'):
        no_params = 19
    param_values = np.zeros((1000, no_params))
    loss_values = []; net_toa_metric = []; loss_after_toa_values = []
    file_path = os.path.join(log_path, file_name)
    print_bool = False
    eval_count = 0
    with open(file_path, 'r') as file:
        for line in file:
            if print_bool:
                for param_ind in range(no_params):
                    # Append certain parameter value
                    param_values[eval_count, param_ind] = float(line.split(',')[param_ind])
                print_bool = False
                eval_count += 1
            # Append line after "Current list of parameters to tune"
            if line.startswith("Current list of parameters to tune"):
                print_bool = True
            if line.startswith("Loss after adding clt_SE_pac:"):
                loss_values.append(float(line.split(':')[1]))
            if line.startswith("Loss after adding net_toa:"):
                loss_after_toa_values.append(float(line.split(':')[1]))
            # TOA value
            if line.startswith("Metric net_toa:"):
                net_toa_metric.append(float(file.readline()[2:-2]))
    return param_values[:len(loss_values), :], np.array(loss_values), np.array(loss_after_toa_values), np.array(net_toa_metric)

For ICON-A-MLe

In [8]:
log_path_iconaml = "/work/bd1083/b309170/published_code/grundner25pnas_iconaml_automatic_tuning/tuning_scripts/auto_tune_iconaml"

In [9]:
# First year-long tuning. Read parameter values and corresponding losses
param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune.12956787.o", log_path_iconaml)

# Index where loss is minimal.
min_ind = np.argmin(loss_values)

# Adjust initial guess by adding a weighted difference, motivated by reducing loss_after_toa_values
C = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[min_ind])

# Those are the parameters I call refined_month_240925, however the second parameter was rounded to 76!!
param_values[0] + C*(param_values[min_ind] - param_values[0])

array([4.09992098e+01, 7.64343601e+01, 2.34842319e+01, 1.32955979e+00,
       3.56392963e-01, 4.40014986e+00, 1.15341387e+01, 2.57516328e+00,
       7.58520407e-01, 6.64995145e-01, 1.01689709e+00, 1.98725558e-04,
       2.41797118e-04, 3.49049307e-04, 1.98434815e+00, 1.03586510e+01,
       9.15158231e-01, 8.30158226e-01, 3.81805561e-01, 8.89074192e-01,
       1.90841672e+01, 2.16284838e+02, 2.09014571e+01, 4.20377280e+01])

In [10]:
# Second year-long tuning. Are the final reported values the ones that minimize the loss in the last script?
param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune.12962670.o", log_path_iconaml)

# Index where loss is minimal
min_ind = np.argmin(loss_values)

great_toa_inds = np.argwhere((0.5 < net_toa_metric) & (net_toa_metric < 1))

# Index for which TOA is great
if len(great_toa_inds) >= 1:
    print('Great TOA at the expense of the loss')
    
    # Extract index with best loss out of the ones with great TOA
    best_loss = np.squeeze(great_toa_inds[np.argmin(loss_values[great_toa_inds])])

    # Prefer parameters with a perfect TOA even if it slightly hurts the loss
    if (loss_values[best_loss] - loss_values[min_ind])/loss_values[min_ind] < 0.25:
        print('Taking the parameter setting with higher loss but better TOA balance')
        final_parameters = param_values[best_loss]
    else:
        final_parameters = param_values[min_ind]
else:
    final_parameters = param_values[min_ind]
    best_loss = min_ind

# Final set of parameters
final_parameters

# Set of parameters I've used for the simulation in the paper. See here: https://github.com/EyringMLClimateGroup/grundner25pnas_iconaml_automatic_tuning/blob/main/simulation_scripts_and_evaluation/auto_tuned_iconaml_full/exp.ag_atm_amip_r2b5_cvtfall_entrmid_05_cov15_based_on_12962670_20yrs_240927.run
paper_parameters = [40.999209779803124, 79.8, 23.484231887639147, 1.3295597852519498, 0.35639296308199575, 4.400149863408263, 11.534138730085267, 2.5751632819728894, 0.7585204066979654, 0.6649951451999998, 1.0168970920619893,\
                    0.00019872555789959162, 0.00024179711798483726, 0.0003490493071823422, 1.9843481475548161, 10.358651006058587, 0.9151582312253206, 0.8301582264050795, 0.38180556104206254,\
                    0.8890741916120648, 19.084167204527887, 216.2848380476869, 20.901457054208564, 42.03772802962055]

print('Associated loss: %.3f'%loss_values[best_loss])

# They are the same
paper_parameters - final_parameters

Great TOA at the expense of the loss
Taking the parameter setting with higher loss but better TOA balance
Associated loss: 1.773


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

For ICON-A

In [11]:
log_path_icona = "/work/bd1083/b309170/published_code/grundner25pnas_iconaml_automatic_tuning/tuning_scripts/auto_tune_icona_baseline"

In [12]:
# First year-long tuning. Read parameter values and corresponding losses
param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.13852199.o", log_path_icona)

# Index where loss is minimal.
min_ind = np.argmin(loss_values)

# Adjust initial guess by adding a weighted difference, motivated by reducing loss_after_toa_values
C = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[min_ind])

# With the same logic as above
param_values[0] + C*(param_values[min_ind] - param_values[0])

array([1.00013844e+00, 5.95913147e-01, 7.01698320e-01, 2.16944655e+00,
       2.48621838e-01, 1.05889486e+00, 2.18095141e-04, 2.05913392e-04,
       3.98469210e-04, 2.18646378e+00, 1.66970491e+01, 8.45743113e-01,
       8.64875152e-01, 3.89880022e-01, 8.32452621e-01, 2.09874253e+01,
       1.77609917e+02, 2.02055062e+01, 8.19499853e+01])

In [13]:
# Second year-long tuning. Are the final reported values the ones that minimize the loss in the last script?
param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.17820065.o", log_path_icona)

# Index where loss is minimal
min_ind = np.argmin(loss_values)

great_toa_inds = np.argwhere((0.5 < net_toa_metric) & (net_toa_metric < 1))

# Index for which TOA is great
if len(great_toa_inds) >= 1:
    print('Great TOA at the expense of the loss')
    
    # Extract index with best loss out of the ones with great TOA
    best_loss = np.squeeze(great_toa_inds[np.argmin(loss_values[great_toa_inds])])

    # Prefer parameters with a perfect TOA even if it slightly hurts the loss
    if (loss_values[best_loss] - loss_values[min_ind])/loss_values[min_ind] < 0.25:
        print('Taking the parameter setting with higher loss but better TOA balance')
        final_parameters = param_values[best_loss]
    else:
        final_parameters = param_values[min_ind]
else:
    final_parameters = param_values[min_ind]
    best_loss = min_ind

# Final loss
print('Final loss: %.3f'%loss_values[best_loss])
print('Final TOA loss: %.3f'%loss_after_toa_values[best_loss])

# Final set of parameters
final_parameters

Final loss: 3.777
Final TOA loss: 0.805


array([9.90000000e-01, 5.95913147e-01, 7.01698320e-01, 2.16944655e+00,
       2.48621838e-01, 1.05889486e+00, 2.18095141e-04, 2.05913392e-04,
       3.98469210e-04, 2.18646378e+00, 1.66970491e+01, 8.88030268e-01,
       8.64875152e-01, 3.89880022e-01, 8.32452621e-01, 2.09874253e+01,
       1.77609917e+02, 2.02055062e+01, 8.19499853e+01])

In [14]:
param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.13887792.o", log_path_icona)

# Index where loss is minimal
min_ind = np.argmin(loss_values)

great_toa_inds = np.argwhere((0.5 < net_toa_metric) & (net_toa_metric < 1))

# Index for which TOA is great
if len(great_toa_inds) >= 1:
    print('Great TOA at the expense of the loss')
    
    # Extract index with best loss out of the ones with great TOA
    best_loss = np.squeeze(great_toa_inds[np.argmin(loss_values[great_toa_inds])])

    # Prefer parameters with a perfect TOA even if it slightly hurts the loss
    if (loss_values[best_loss] - loss_values[min_ind])/loss_values[min_ind] < 0.25:
        print('Taking the parameter setting with higher loss but better TOA balance')
        final_parameters = param_values[best_loss]
    else:
        final_parameters = param_values[min_ind]
else:
    final_parameters = param_values[min_ind]
    best_loss = min_ind

print('Loss after tweaking: %.3f'%loss_values[best_loss])
print('TOA loss after tweaking: %.3f'%loss_after_toa_values[best_loss])

# Final parameters
param_values[min_ind]

Great TOA at the expense of the loss
Taking the parameter setting with higher loss but better TOA balance
Loss after tweaking: 3.521
TOA loss after tweaking: 0.000


array([9.68000000e-01, 6.65000000e-01, 7.01698320e-01, 2.16944655e+00,
       2.61052930e-01, 1.05889486e+00, 2.18095141e-04, 2.05913392e-04,
       3.98469210e-04, 2.18646378e+00, 1.66970491e+01, 8.45743113e-01,
       8.64875152e-01, 3.89880022e-01, 8.32452621e-01, 2.09874253e+01,
       1.77609917e+02, 2.02055062e+01, 8.19499853e+01])

-------

In [15]:
# # Loss after re-running year-long simulations after the tweak
# param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.13887792.o", log_path_icona)

# min_ind = np.argmin(loss_values)
# great_toa_inds = np.argwhere((0.5 < net_toa_metric) & (net_toa_metric < 1))

# # Index for which TOA is great
# if len(great_toa_inds) >= 1:
#     print('Great TOA at the expense of the loss')
    
#     # Extract index with best loss out of the ones with great TOA
#     best_loss = np.squeeze(great_toa_inds[np.argmin(loss_values[great_toa_inds])])

#     # Prefer parameters with a perfect TOA even if it slightly hurts the loss
#     if (loss_values[best_loss] - loss_values[min_ind])/loss_values[min_ind] < 0.25:
#         print('Taking the parameter setting with higher loss but better TOA balance')
#         final_parameters = param_values[best_loss]
#     else:
#         final_parameters = param_values[min_ind]
# else:
#     final_parameters = param_values[min_ind]

# print('Loss after tweaking and rerunning: %.3f'%loss_values[best_loss])
# print('TOA loss after tweaking and rerunning: %.3f'%loss_after_toa_values[best_loss])

# paper_parameters = np.array([9.68000000e-01, 6.98250000e-01, 7.01698320e-01, 2.16944655e+00,
#        2.48621838e-01, 1.05889486e+00, 2.18095141e-04, 2.05913392e-04,
#        3.98469210e-04, 2.18646378e+00, 1.66970491e+01, 8.45743113e-01,
#        8.64875152e-01, 3.89880022e-01, 8.32452621e-01, 2.09874253e+01,
#        1.77609917e+02, 2.02055062e+01, 8.19499853e+01])

# # Final parameters
# final_parameters

# final_parameters - paper_parameters

In [170]:
# # Adjust initial guess by adding a weighted difference, motivated by reducing loss_after_toa_values
# C1 = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[1])
# C2 = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[2])

# alpha1 = 0.63 # Not well motivated
# alpha2 = 1-alpha1

# final_parameters = param_values[0] + C1*(param_values[1] - param_values[0])*alpha1 + C2*(param_values[2] - param_values[0])*alpha2
# final_parameters

In [171]:
# # Set of parameters I've used for the simulation in the paper. See here: https://github.com/EyringMLClimateGroup/grundner25pnas_iconaml_automatic_tuning/blob/main/simulation_scripts_and_evaluation/auto_tuned_baseline/exp.ag_atm_amip_r2b5_auto_tuned_baseline_20yrs.run
# paper_parameters = [0.968, 0.69825, 0.7016983204535419, 2.169446548166456, 0.24862183830475587, 1.0588948615278229, 0.00021809514111887957, 0.000205913392405527, 0.00039846920997366024,\
#                     2.186463783358046, 16.697049053748103, 0.8457431126829968, 0.8648751522165565, 0.3898800218636592, 0.8324526207821763, 20.98742526609769, 177.60991737322595,\
#                     20.20550621794748, 81.94998525907712]

# final_parameters - paper_parameters

In [172]:
# # Adjust initial guess by adding a weighted difference, motivated by reducing loss_after_toa_values
# C1 = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[1])
# C2 = loss_after_toa_values[0]/(loss_after_toa_values[0] - loss_after_toa_values[2])

# # alpha = 1/2 is well motivated
# alpha1 = 0.5
# alpha2 = 0.5

# final_parameters = param_values[0] + C1*(param_values[1] - param_values[0])*alpha1 + C2*(param_values[2] - param_values[0])*alpha2
# final_parameters

In [173]:
# # Results of intermediary strategies:
# log_path_icona = "/work/bd1179/b309170/icon-ml_models/icon-a-ml/run"
# # 17820065 (year_refined_5) 
# param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.17820065.o", log_path_icona)
# # Best parameter setting
# min_ind = np.argmin(loss_values)
# # Parameter that has actually changed
# diff_par = np.argmax(param_values[min_ind] - param_values[0])
# print('Verdict of year_refined_5:')
# print('Changing the %dth parameter from %.3f to %.3f helps most.'%(diff_par+1, param_values[0,diff_par], param_values[min_ind,diff_par]))

# print(loss_values[min_ind])

# # 17821143 (year_refined_6) 
# param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.17821143.o", log_path_icona)
# # Best parameter setting
# min_ind = np.argmin(loss_values)
# # Parameter that has actually changed
# diff_par = np.argmax(param_values[min_ind] - param_values[0])
# print('Verdict of year_refined_6:')
# print('Changing the %dth parameter from %.3f to %.3f helps most.'%(diff_par+1, param_values[0,diff_par], param_values[min_ind,diff_par]))

# print(loss_values[min_ind])

# # 17821894 (year_refined_7) 
# param_values, loss_values, loss_after_toa_values, net_toa_metric = create_list("log.auto_tune_baseline.17821894.o", log_path_icona)
# # Best parameter setting
# min_ind = np.argmin(loss_values)
# # Parameter that has actually changed
# diff_par = np.argmax(param_values[min_ind] - param_values[0])
# print('Verdict of year_refined_7:')
# print('Changing the %dth parameter from %.3f to %.3f helps most.'%(diff_par+1, param_values[0,diff_par], param_values[min_ind,diff_par]))

# print(loss_values[min_ind])